In [8]:
import torch
from torchvision.models import convnext_tiny, ConvNeXt_Tiny_Weights
import torch.nn as nn
class LayerNorm2d(nn.LayerNorm):
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.permute(0, 2, 3, 1)
        x = torch.nn.functional.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        x = x.permute(0, 3, 1, 2)
        return x
n_inputs = 768
hidden_size = 256 # default is 2048
n_outputs = 10

model = convnext_tiny(weights=ConvNeXt_Tiny_Weights.DEFAULT)
sequential_layers = nn.Sequential(
    LayerNorm2d((1024,), eps=1e-06, elementwise_affine=True),
    nn.Flatten(start_dim=1, end_dim=-1),
    nn.Linear(n_inputs, hidden_size, bias=True),
    nn.BatchNorm1d(hidden_size),
    nn.ReLU(),
    nn.Dropout(0.1),
    nn.Linear(hidden_size, hidden_size),
    nn.BatchNorm1d(hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, n_outputs),
    nn.LogSoftmax(dim=1)
)
model.classifier = sequential_layers
'''
layers = [(name, param.requires_grad) for name, param in model.named_parameters()]
for i in range(len(layers)):
    print(layers[i])
'''
'''
n_inputs = None
for name, child in model.named_children():
    if name == 'classifier':
        for sub_name, sub_child in child.named_children():
            if sub_name == '2':
                n_inputs = sub_child.in_features
'''
num = 0
for name, param in model.named_parameters():
    if (param.requires_grad):
        num += 1
    #if (num > 159): # for freezing layers 1, 2, 3, and 4
    #if (num > 129): # for freezing layers 1, 2, and 3
    #if (num > 72): # for freezing layers 1 and 2
    #if (num > 33): #for freezing layer 1
    #if (num >= 0): # for no freezing
        #param.requires_grad = True
    #else:
        #param.requires_grad = False
print(num)

190
